# Import everything we need

In [16]:
import pandas as pd
import numpy as np
import requests
import json
import os
import psycopg2 # Needed to get database errors when uploading dataframe
import sql
import sqlalchemy
import warnings # hides warning messages
warnings.filterwarnings("ignore")

## Create dictionary for competitors & locations

In [15]:
search_term = { 

'Air France' : ['Mexico', 'Netherlands', 'China', 'Vietnam', 'Thailand', 'Indonesia', 
                'Malaysia', 'Morocco', 'Tunisia', 
                'France'
                ],

'Delta TechOps' : ['United States', 'Canada', 'Mexico', 'Argentina', 'Bolivia', 'Brazil', 'Chile', 
                    'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 
                    'Peru', 'South Georgia', 
                    'Suriname', 'Uruguay', 'Venezuela'],
                   
'MTU Maintenance' : ['China', 'Serbia', 'Poland', 'Canada', 'United States', 'Malaysia', 'Germany'],

'Lufthansa' : ['Germany', 'Hungary', 'Bulgaria'],

'Safran' : ['Belgium', 'France', 'Dubai', 'Mexico', 'United Kingdom', 'Germany']
}

# Looped request from API

In [ ]:
import time
from functions_for_jobs import clean_location 
from functions_for_jobs import create_city_state_country 
from functions_for_jobs import assign_world_regions
from functions_for_jobs import create_id
from functions_for_jobs import translate_job_titles
from functions_for_jobs import clean_job_titles
from functions_for_jobs import clean_text
from functions_for_jobs import translate_job_description
from functions_for_jobs import delete_columns



url = "https://linkedin-jobs-search.p.rapidapi.com/"
headers = {
            "content-type": "application/json",
            "X-RapidAPI-Host": "linkedin-jobs-search.p.rapidapi.com",
            "X-RapidAPI-Key": "cfcb3737afmshd6f1a91c82f4d4cp1d9823jsn3d33e41d670f"
            }
i = 1
for competitor in search_term:  

    for search_country in search_term[competitor]:
        
        while i < 15:
### Getting Data via API    
            payload = {
                        "search_terms": f'{competitor}',
                        "location": f'{search_country}', 									        
                        "page": f'{i}',     										        
                        "fetch_full_text": "yes"
                        }
           
            response = requests.request("POST", url, json=payload, headers=headers)
            jobs_list = json.loads(response.content)
            json.dumps(jobs_list, indent=3)
            job = pd.json_normalize(jobs_list, sep="_")
            print(job)
            print(competitor)
            print(search_country)
            time.sleep(15)   
            print('slept well and ready to go on')                                                                      # just 5 requests per minute are allowed
        

            if len(job) > 1:         
### Cleaning
                
                jobs = job[['company_name', 'job_title','job_location','posted_date','full_text']] # create DataFrame with necessary columns
                jobs['search_country'] = search_country              
                jobs = clean_location(jobs)
                jobs = create_city_state_country(jobs)
                jobs = assign_world_regions(jobs)
                jobs = create_id(jobs)
                jobs = translate_job_titles(jobs)
                jobs = clean_job_titles(jobs)
                jobs = clean_text(jobs)
                jobs['posted_date'] = pd.to_datetime(jobs['posted_date'], format="%Y-%m-%d")
                jobs = translate_job_description(jobs)
                jobs = delete_columns(jobs)
                
                
                
### Upload
                schema = os.getenv('schema')
                from sql import engine
                for row in range(len(jobs)):
                    try:
                        jobs.iloc[row:row+1, :].to_sql(name='jobs',                     # Name of DF table
                                                        con=engine,                     # Engine or connection
                                                        if_exists='append',             # Just add new values in existing table
                                                        schema=schema,                  # Use schema that was defined earlier in .sql
                                                        index=False,                    # Write DataFrame index as a column
                                                        chunksize=1,                    # Specify the number of rows in each batch to be written at a time
                                                        method='multi')                 # Pass multiple values in a single INSERT clause
                        print(f"row {row} of jobs was inserted successfully.")
                    except:
                        print(f"row {row} of jobs was ignored.")
                        continue
                i += 1 
                              
            else:
                i=1
                break

# Single Request from API

search_term, location and page have to be assigned manually

In [ ]:
url = "https://linkedin-jobs-search.p.rapidapi.com/"
headers = {
            "content-type": "application/json",
            "X-RapidAPI-Host": "linkedin-jobs-search.p.rapidapi.com",
            "X-RapidAPI-Key": "a6d3c92a01msh4212f13f83cf821p182fedjsn1bd153fd656a"
            }
### Getting Data via API    
payload = {
            "search_terms": 'aeronautical engineer',
            "location": '', 									        
            "page": '1',     										        
            "fetch_full_text": "yes"
            }

response = requests.request("POST", url, json=payload, headers=headers)
jobs_list = json.loads(response.content)
json.dumps(jobs_list, indent=3)
job = pd.json_normalize(jobs_list, sep="_")
                                                                    # just 5 requests per minute are allowed

jobs_single = job[['company_name', 'job_title','job_location','posted_date','full_text']] # create DataFrame with necessary columns